In [28]:
import pandas as pd
from cassandra.cluster import Cluster

cluster = Cluster(['192.168.7.110', '192.168.7.111'])
#session = cluster.connect('data.storageinfo')
session = cluster.connect()

rows = session.execute('select loc from data.storageinfo limit 5')
for row in rows:
    print(row.loc.split('\u0001'))
    #print(type(row.loc))
    #print(row.loc)

cluster.shutdown()

['192.168.7.71', '80', '/baseline_20181012/form10.pdf', 'HTTP-F5yME82fRLXCROAPsa.pdf', 'DATE_OF_BIRTH']
['192.168.7.71', '80', '/baseline_20181012/form6.pdf', 'HTTP-Frv0wK3ezzMAzGyU39.pdf', 'SOCIAL_SECURITY_NUMBER']
['192.168.8.71', '80', '/PIIFormTest/blank_form1.pdf', 'HTTP-FAVen34Mq8fv2iY4Y.pdf', 'STREET_NUMBER_AND_NAME']
['192.168.7.71', '80', '/baseline_20181012/form6.pdf', 'HTTP-FqOYpu10gI4KtX0dBc.pdf', 'EMAIL']
['192.168.8.71', '8080', '/form1.pdf', 'HTTP-FSknBM1IoeT9f0FHIa.pdf', 'DATE_OF_BIRTH']


In [3]:
from cassandra.cluster import Cluster

def cs_select(cs_host_list, csql_str):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    res = session.execute(csql_str)
    cluster.shutdown()
    return res
    
#rows = cs_select(['192.168.7.110', '192.168.7.111'], "select * from api.ds_services where ip='192.168.8.74' and port=1433")
#[print(row.ip, type(row.files), type(row.badges), row.connection, row.port) for row in rows]

DS_DELIMITER = "\u0001"
sel_ip = "192.168.7.71"
sel_port = 1433
sel_l = DS_DELIMITER.join([sel_ip, str(sel_port)]) + DS_DELIMITER
print(sel_l)
sel_u = DS_DELIMITER.join([sel_ip, str(sel_port+1)]) + DS_DELIMITER
print(sel_u)

#query = "select * from scanner.sqlcatalog"# where loc LIKE '%192.168.7.71%'"
#rows = cs_select(['192.168.7.21', '192.168.7.22'], query)
#[':'.join([row.db_name, row.table_name, row.col_name]) for row in rows if sel in row.loc]

query = "select * from scanner.sqlcatalog where loc >= '" + sel_l + "' and loc < '" + sel_u + "' allow filtering"
rows = cs_select(['192.168.7.21', '192.168.7.22'], query)
[':'.join([row.ip, str(row.port), row.db_name, row.table_name, row.col_name]) for row in rows]

#rows = cs_select(['192.168.7.110', '192.168.7.111'], 'select sum(dbs) as dbs, sum(tables) as tables, sum(files) as files from api.ds_services')
#[row.dbs for row in rows][0]

#rows = cs_select(['192.168.7.110', '192.168.7.111'], 'select count(ip) as servers, sum(services) as services from api.ds_servers')
#[row.servers for row in rows][0]
#servers, services = [(row.servers, row.services) for row in rows][0]
#print(servers, services)
#[print(type(row.ip)) for row in rows]

192.168.7.711433
192.168.7.711434


[]

In [49]:
query = "select ddn_name, city, count(pii_record) as value from api.content_locations group by ddn_name, city"
query = "select ddn_name, city, count(last_seen_time) as value from api.access_locations group by ddn_name, city"
#query = "select city, pii_record from api.content_locations where ddn_name = 'aaa' allow filtering"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"ddn_name": row.ddn_name,
            "content": row.city,
            "value": row.value} for row in rows]
#print(cs_data)
#if cs_data:
#    print("aaa")
#else:
#    print("bbb")
cs_data = [
    {"ddn_name": "hr", "content": "Beijing", "value": 5},
    {"ddn_name": "hr", "content": "Tianjin", "value": 1},
    {"ddn_name": "sales", "content": "Shanghai", "value": 5},
    {"ddn_name": "sales", "content": "Beijing", "value": 22},
    {"ddn_name": "hr", "content": "Nanjing", "value": 1},
]

df = pd.DataFrame(cs_data)
d = df.groupby(['content'])['value'].sum().to_dict()
cs_data = [{"content": k, "value": v} for k, v in d.items()]
cs_data

[{'content': 'Beijing', 'value': 27},
 {'content': 'Nanjing', 'value': 1},
 {'content': 'Shanghai', 'value': 5},
 {'content': 'Tianjin', 'value': 1}]